In [2]:
import numpy as np
import pandas as pd

In [3]:
hindi_df = pd.read_csv('/content/combined_cleaned_hindi_12037_df.csv')
hindi_df.head()

,Text_description
0,परधनमतर नरदर मद अमरक रषटरपत ज बइडन क बच पछल दन...
1,मर पत चहत थ क म हमश तयर सज सवर रह म मथ बड बद ल...
2,गजरत म लगतर ह रह भर बरश क करण जल म बढ क सथत बन...
3,अवन लखर परस परलपक म महलओ क मटर एयर रइफल सटडग स...
4,परस परलपक खल क शरआत अगसत स ह चक ह ओपनग सरमन म ...


In [4]:
telugu_df = pd.read_csv('/content/combined_cleaned_telugu_10628_df.csv')
telugu_df.head()

,cleaned_text
0,ట్రంప్‌ సుంకాల నేపథ్యంలో ఆర్థికమంత్రి నిర్మలా ...
1,"భారత మార్కెట్లు ట్రంప్‌ సుంకాల విరామం, వాణిజ్య..."
2,ఎన్‌ఎస్ఈ ఐపీఓ అనుమతిలో వాణిజ్య ప్రయోజనాల కంటే ...
3,ఐటీసీ గ్రూప్‌ 24 మంత్ర ఆర్గానిక్‌ బ్రాండ్‌ను ర...
4,టాటా మోటార్స్‌ 2024-25 ఆర్థిక సంవత్సరంలో 250 ప...


In [5]:
hindi_df.shape

(12037, 1)

In [6]:
hindi_df.isnull().sum()

,0
Text_description,0


In [8]:
hindi_duplicates = hindi_df[hindi_df.duplicated()]
print(f"Number of duplicates: {hindi_duplicates.shape[0]}")

Number of duplicates: 0


In [9]:
telugu_df.shape

(10628, 1)

In [10]:
telugu_df.isnull().sum()

,0
cleaned_text,0


In [12]:
telugu_duplicates = telugu_df[telugu_df.duplicated()]
print(f"Number of duplicates: {telugu_duplicates.shape[0]}")

Number of duplicates: 0


# Lasya_Lasya_Lasya_Lasya_Lasya_Lasya

In [13]:
import pandas as pd
import re
import os

# 1. Load your full cleaned datasets
hindi_path  = '/content/combined_cleaned_hindi_12037_df.csv'
telugu_path = '/content/combined_cleaned_telugu_10628_df.csv'
hindi_df    = pd.read_csv(hindi_path)
telugu_df   = pd.read_csv(telugu_path)

# 2. Identify the text columns
hindi_text_col  = 'Text_description'
telugu_text_col = 'cleaned_text'

# 3. Load + dedupe the NRC lexicons
hindi_lex  = pd.read_csv('/content/Hindi-NRC-EmoLex.txt', sep='\t').drop_duplicates('Hindi Word')
telugu_lex = pd.read_csv('/content/Telugu-NRC-EmoLex.txt', sep='\t').drop_duplicates('Telugu Word')

# 4. Precompute a lookup map for speed
emotions = ['anger','anticipation','disgust','fear','joy',
            'negative','positive','sadness','surprise','trust']

hindi_map = {
    row['Hindi Word'].lower(): {emo: row[emo] for emo in emotions}
    for _, row in hindi_lex.iterrows()
}
telugu_map = {
    row['Telugu Word'].lower(): {emo: row[emo] for emo in emotions}
    for _, row in telugu_lex.iterrows()
}

# 5. Define a fast annotator
def dominant_emotion(text, lex_map):
    counts = dict.fromkeys(emotions, 0)
    for w in re.findall(r'\w+', str(text).lower()):
        if w in lex_map:
            for emo, val in lex_map[w].items():
                counts[emo] += val
    # pick the highest‐scoring emotion (or 'neutral' if all zero)
    return max(counts, key=counts.get) if any(counts.values()) else 'neutral'

# 6. Apply to every row
hindi_df['emotion_label']  = hindi_df[hindi_text_col].fillna('').map(lambda t: dominant_emotion(t, hindi_map))
telugu_df['emotion_label'] = telugu_df[telugu_text_col].fillna('').map(lambda t: dominant_emotion(t, telugu_map))

# 7. Save the fully annotated sets
#hindi_df.to_csv('/mnt/data/combined_cleaned_hindi_annotated_full.csv', index=False)
#telugu_df.to_csv('/mnt/data/combined_cleaned_telugu_annotated_full.csv', index=False)

#print("Done! Annotated files written to:")
#print(" - /mnt/data/combined_cleaned_hindi_annotated_full.csv")
#print(" - /mnt/data/combined_cleaned_telugu_annotated_full.csv")


In [14]:
hindi_df.head()

,Text_description,emotion_label
0,परधनमतर नरदर मद अमरक रषटरपत ज बइडन क बच पछल दन...,positive
1,मर पत चहत थ क म हमश तयर सज सवर रह म मथ बड बद ल...,positive
2,गजरत म लगतर ह रह भर बरश क करण जल म बढ क सथत बन...,positive
3,अवन लखर परस परलपक म महलओ क मटर एयर रइफल सटडग स...,anger
4,परस परलपक खल क शरआत अगसत स ह चक ह ओपनग सरमन म ...,negative


In [15]:
hindi_df.shape

(12037, 2)

In [16]:
telugu_df.head()

,cleaned_text,emotion_label
0,ట్రంప్‌ సుంకాల నేపథ్యంలో ఆర్థికమంత్రి నిర్మలా ...,neutral
1,"భారత మార్కెట్లు ట్రంప్‌ సుంకాల విరామం, వాణిజ్య...",neutral
2,ఎన్‌ఎస్ఈ ఐపీఓ అనుమతిలో వాణిజ్య ప్రయోజనాల కంటే ...,neutral
3,ఐటీసీ గ్రూప్‌ 24 మంత్ర ఆర్గానిక్‌ బ్రాండ్‌ను ర...,neutral
4,టాటా మోటార్స్‌ 2024-25 ఆర్థిక సంవత్సరంలో 250 ప...,neutral


In [17]:
telugu_df.shape

(10628, 2)

In [18]:
emotion_attached_hindi = hindi_df
emotion_attached_hindi

,Text_description,emotion_label
0,परधनमतर नरदर मद अमरक रषटरपत ज बइडन क बच पछल दन...,positive
1,मर पत चहत थ क म हमश तयर सज सवर रह म मथ बड बद ल...,positive
2,गजरत म लगतर ह रह भर बरश क करण जल म बढ क सथत बन...,positive
3,अवन लखर परस परलपक म महलओ क मटर एयर रइफल सटडग स...,anger
4,परस परलपक खल क शरआत अगसत स ह चक ह ओपनग सरमन म ...,negative
...,...,...
12032,पीसीएएफ ने वित्तीय उद्योग के लिए वैश्विक जीएच...,neutral
12033,पीसीएएफ ने वित्तीय उत्सर्जन रिपोर्टिंग मानक मे...,neutral
12034,"कार्टर ने लॉन्च कि ईएसजी रणनीति, जिसमें स्थायी...",neutral
12035,गेस्ट पोस्ट: नेट शून्य वित्त हो सकता है उत्तर ...,neutral


In [19]:
emotion_attached_telugu = telugu_df
emotion_attached_telugu

,cleaned_text,emotion_label
0,ట్రంప్‌ సుంకాల నేపథ్యంలో ఆర్థికమంత్రి నిర్మలా ...,neutral
1,"భారత మార్కెట్లు ట్రంప్‌ సుంకాల విరామం, వాణిజ్య...",neutral
2,ఎన్‌ఎస్ఈ ఐపీఓ అనుమతిలో వాణిజ్య ప్రయోజనాల కంటే ...,neutral
3,ఐటీసీ గ్రూప్‌ 24 మంత్ర ఆర్గానిక్‌ బ్రాండ్‌ను ర...,neutral
4,టాటా మోటార్స్‌ 2024-25 ఆర్థిక సంవత్సరంలో 250 ప...,neutral
...,...,...
10623,హదరబద కదరగ ఉనన సస ఎటరటనమట లమటడ జయ గమసత వయహతమక ...,neutral
10624,దశ ఆరథక వయవసథ గడనపడతద పరసతత ఆరథక సవతసర డసబ రత ...,neutral
10625,indian students ఇటవల రపయ వలవ డలరత పలసత పడపవడత ...,neutral
10626,gold rates రజ మరపగన బగర ధర కడ మరపత ఉటద ఒకకసర ఉ...,neutral


In [20]:
emotion_counts_hindi = emotion_attached_hindi['emotion_label'].value_counts()
print(emotion_counts_hindi)

emotion_label
negative        3236
neutral         3020
positive        2703
anger           1037
joy              780
anticipation     567
disgust          441
trust            133
surprise          81
fear              32
sadness            7
Name: count, dtype: int64


In [21]:
emotion_counts_telugu = emotion_attached_telugu['emotion_label'].value_counts()
print(emotion_counts_telugu)

emotion_label
neutral         7220
fear            2257
positive         694
anger            318
negative          89
anticipation      37
disgust            9
trust              4
Name: count, dtype: int64


# Lasyamma

In [ ]:
df_annotated_hindi = pd.read_csv('/content/Annotated_cleaned_Hindi_full.csv')
df_annotated_hindi.head()

,Text_description,emotion_label
0,परधनमतर नरदर मद अमरक रषटरपत ज बइडन क बच पछल दन...,positive
1,मर पत चहत थ क म हमश तयर सज सवर रह म मथ बड बद ल...,positive
2,गजरत म लगतर ह रह भर बरश क करण जल म बढ क सथत बन...,positive
3,अवन लखर परस परलपक म महलओ क मटर एयर रइफल सटडग स...,anger
4,परस परलपक खल क शरआत अगसत स ह चक ह ओपनग सरमन म ...,negative


In [ ]:
df_annotated_hindi.shape

(3839, 2)

In [22]:
emotion_attached_telugu.to_csv('emotion_attached_telugu.csv', index=False)
emotion_attached_hindi.to_csv('emotion_attached_hindi.csv', index=False)